# Notebook
## Sistem Rekomendasi Obat berdasarkan _Weighted Hybrid Approach_

Notebook ini menyajikan penerapan dan eksperimentasi _weighted hybrid approach_ pada sistem rekomendasi obat. _Weighted hybrid approach_ yang diusulkan merupakan kombinasi antara .... dan .... . Beberapa teknik pembobotan yang akan dimasukkan dalam eksperimentasi antara lain ...., ...., dan .....

## 1. Pustaka

Impor pustaka yang dibutuhkan.

In [ ]:
# pustaka eksternal
import re
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import xgboost as xgb
from sklearn import preprocessing
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MultiLabelBinarizer

# pustaka custom
from analysis import *

## 2. Persiapan Set Data

Pada bagian ini, dilakukan persiapan set data yang dibutuhkan dalam penelitian. Karena set data yang sesuai untuk secara langsung digunakan dalam penelitian ini belum tersedia saat penelitian ini dikerjakan, maka Peneliti melakukan pengumpulan data dari beberapa sumber dan menggabungkannya.

Set data `WebMD` dan `DailyMed` digabungkan dalam struktur data DataFrame.

In [ ]:
# berkas WebMD Drug Reviews oleh Rohan Harode dari Kaggle
path_wmd = "data/webmd/webmd.csv"

# berkas DailyMed Drug dari situs web DailyMed
# yang dikompilasi dengan menggunakan generate_dailymed.py
path_dm = "data/dailymed/dailymed.csv"

df_wmd = pd.read_csv(path_wmd)
df_dm = pd.read_csv(path_dm)

Atribut yang diperlukan sebagai pelengkap data pada DailyMed, antara lain:
1. nama obat secara umum pada kolom `sub_name`
2. daftar bahan aktif pada kolom `list_activeIngredient`, dan 
3. daftar bahan inaktif pada kolom `list_activeIngredient`.

Oleh karena itu, perlu dilakukan seleksi kolom pada DataFrame `df_dm` sebagai berikut.

In [ ]:
df_dm = df_dm[["SubName", "ListActiveIngredient", "ListInactiveIngredient"]]

Gabungkan DataFrame DailyMed `df_dm` dan DataFrame MebMD `df_wmd` dengan menggunakan kolom `sub_name` di `df_dm` dan kolom `Drug` di `df_wmd` sebagai indeks pengabungan.

In [ ]:
df_drugs = pd.merge(df_dm, df_wmd, left_on = "SubName", right_on = "Drug")

## 3. Analisis Data

Dalam penelitian ini, analisis dilakukan pada `df_drugs` untuk memahami tipe data dan mengidentifikasi kesalahan. Informasi tersebut digunakan untuk menentukan prosedur yang sesuai dalam tahap prapemrosesan.

Kita mulai dengan menghitung jumlah masing-masing nilai yang berbeda pada kolom `Drug` untuk mengetahui jumlah baris data untuk setiap obat. Informasi ini diperlukan untuk memastikan kecukupan data penelitian.

In [ ]:
display(df_drugs["Drug"].value_counts())

Pada sel tersebut, kita dapat melihat bahwa ada 74 nama obat dalam `df_drugs`. Obat dengan baris data terbanyak adalah `lisinopril` (12.807), sementara obat dengan baris data tersedikit adalah `cromolyn sodium`, `caffeine citrate`, dan `bexarotene` (2).

Jika menggunakan 500 baris data sebagai ambang batas kecukupan data, maka persentase jumlah obat disajikan sebagai berikut.

### Tipe data

In [ ]:
df_drugs.dtypes

Untuk memeriksa tipe data dari masing2 varibael/kolom/fitur dengan menggunakan .dtypes. untuk dtype (object) pada pandas digunakan untuk tipe data Teks atau campuran nilai numerik dan non-numeric. untuk dtype (int64) pada pandas digunakan untuk tipe data Integer numbers. Pada sel diatas, kita dapat melihat bahwa 2 jenis type data yang muncul yaitu object dan int64.

In [ ]:
for col in df_drugs:
    drugs = pd.get_dummies(df_drugs[col],prefix='', prefix_sep='').sum() 
    print(col, ":", drugs)

Pada sel diatas, kita dapat melihat rentang nilai dari setiap atribut. 

|     Nama attribut      |      Tipe Attribut     |                Rentang Nilai                                    |
| ---------------------- | -----------------------| ----------------------------------------------------------------|
|      SubName           |        Nominal         |  acarbose, acetazolamide , aripiprazole, baclofen, benzonatate,...,valproic acid, voriconazole, zaleplon, zolmitriptan, zolmitriptan, zonisamide   |
| ListActiveIngredient   |        Nominal         |  ACARBOSE 25 mg -1-1, ACETAZOLAMIDE 125 mg -1-1, ACETAZOLAMIDE 250 mg -1-1, ACETAZOLAMIDE 500 mg -1-1, ARIPIPRAZOLE1 mg 1mL, ...,valproic acid 250 mg -1-1 |
| ListInactiveIngredient |        Nominal         |  ACETIC ACID, SODIUM CHLORIDE, SODIUM ACETATE, WATER, ALCOHOL, ANHYDROUS CITRIC ACID, ... sodium hydroxide, hydrochloric acid|
|       Age              |        Nominal         |  0-2, 13-18, 19-24, 25-34, 3-6, 35-44, 45-54, 55-64, 65-74, 7-12, 75 or over|
|     Condition          |        Nominal         |  "Change of Life" Signs, A Condition of Bladder Dysfunction from Nerve Disorder , A Condition of Bladder Dysfunction from Nerve Disorder, ..., infection caused by bacteria    |
|     Date               |        Nominal         | 1/1/2008, 1/1/2009, 1/1/2011, 1/1/2012, ..., 9/9/2013 9/9/2017         |
|     Drug               |        Nominal         | acarbose, acetazolamide, aripiprazole, baclofen, benzonatate, ... zonisamide |
|     DrugId             |        Nominal         | 676, 911, 1027, 1049, 1636, ... ,93290,  94574, 148989,  149812  |
|     EaseofUse          |       Ordinal          |  1, 2, 3, 4, 5                                                   |
|     Effectiveness      |       Ordinal          |  1, 2, 3, 4, 5                                                   |
|    Reviews             |       Nominal          |  an this drug be taken on a long term bases? I had a cough for years that I dr with and nothing worked. I was told it can not be taken long term. What is the side effects if taken long term, ......   a 90 day supply of Lisinopril 10 mg which averages $3.33 per month. |
|     Satisfaction       |       Ordinal          |  1, 2, 3, 4, 5                                                   |
|     Sex                |       Binary           |  Female, Male                                                    |
|     Sides              |       Nominal          |  Constipation ,  nausea ,  headache ,  diarrhea ,  vomiting ,  stomach  upset, gas, tremor,  dizziness , drowsiness, or  trouble sleeping  may occur, ... Upset stomach ,  nausea ,  vomiting , gas, or  diarrhea  may occur |
|     UsefulCount        |       Ordinal          |  0, 1, 2, 3, 4...... 140                                          |


### Mengidentifikasi Kesalahan

In [ ]:
df_drugs.isnull().sum()

Dalam mengidentifikasi kesalahahan dapat dilakukan agregasi data menggunakan fungsi sum(). Berdasarkan analisis diatas dapat dilihat bahwa dalam dataset tersebut hanya ada satu atribut yang memiliki nilai kosong atau NULL sebanyak 16 missing value.

In [ ]:
s = pd.get_dummies(df_drugs['SubName'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["SubName"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut sub_name tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field sub_name 

In [ ]:
s = pd.get_dummies(df_drugs['ListActiveIngredient'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["ListActiveIngredient"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut list_activeIngredient tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field list_activeIngredient

In [ ]:
s = pd.get_dummies(df_drugs['ListInactiveIngredient'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["ListInactiveIngredient"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut ListInactiveIngredient tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field ListInactiveIngredient

In [ ]:
s = pd.get_dummies(df_drugs['Age'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["Age"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut 'Age' terdapat nilai kosong atau missing value sebanyak 1941 

In [ ]:
s = pd.get_dummies(df_drugs['Condition'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["Condition"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut Condition tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field 'Condition'

In [ ]:
s = pd.get_dummies(df_drugs['Date'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["Date"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut 'Date' tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field 'Date'

In [ ]:
s = pd.get_dummies(df_drugs['Drug'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["Drug"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut 'Drug' tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field 'Drug'

In [ ]:
s = pd.get_dummies(df_drugs['EaseofUse'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["EaseofUse"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut 'EaseofUse' tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field 'EaseofUse'

In [ ]:
s = pd.get_dummies(df_drugs['Effectiveness'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["Effectiveness"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut 'Effectiveness' tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field 'Effectiveness'

In [ ]:
s = pd.get_dummies(df_drugs['Satisfaction'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["Satisfaction"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut 'Satisfaction' tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field 'Satisfaction'

In [ ]:
s = pd.get_dummies(df_drugs['Sex'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["Sex"].value_counts()
for i in item_counts.items():
    print(i) 

Untuk kolom atribut 'Sex' terdapat nilai kosong atau missing value sebanyak 4349

In [ ]:
s = pd.get_dummies(df_drugs['Sides'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["Sides"].value_counts()
for i in item_counts.items():
    print(i)

Untuk kolom atribut 'Sides' tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field 'Sides'

In [ ]:
s = pd.get_dummies(df_drugs['UsefulCount'],prefix='', prefix_sep='').sum() 
item_counts = df_drugs["UsefulCount"].value_counts()
for i in item_counts.items():
    print(i)

Untuk kolom atribut 'UsefulCount' tidak ada nilai aneh atau ganjil serta tidak ditemukan nilai yang kosong pada field 'UsefulCount'

In [ ]:
data1 = df_drugs.drop(columns = ['Reviews'])
for col in data1:
    s = pd.get_dummies(df_drugs[col],prefix='', prefix_sep='').sum() 
    print(col, ":", s)

# s = pd.get_dummies(data1,prefix='', prefix_sep='').sum().sort()
# s

In [ ]:
n_sufficient = sum(x >= 500 for x in df_drugs["Drug"].value_counts())
n_insufficient = sum(x < 500 for x in df_drugs["Drug"].value_counts())
print("Cukup", n_sufficient, "dan", "Tidak Cukup:", n_insufficient)
# fig = go.Figure(data=[go.Pie(labels=["Cukup", "Tidak Cukup"],
#                              values=[n_sufficient, n_insufficient], hole = 0.5)])
# fig.show()

Memperhatikan persentase tersebut, maka diperlukan penambahan lebih banyak baris data untuk setiap obat dengan total baris data yang kurang dari 500. Untuk saat ini, kita cukup gunakan baris data obat yang cukup.

In [ ]:
list_sufficient = [val for val, cnt in df_drugs["Drug"].value_counts().iteritems() if cnt >= 500]
df_drugs_sub = df_drugs[df_drugs["Drug"].isin(list_sufficient)]

In [ ]:
# df_drugs_sub.pivot_table(columns=["Sex", "Age", "Condition"], aggfunc="size")

Periksa banyaknya baris data dengan nilai atribut `Age` dan `Sex` yang kosong.

In [ ]:
cnt_empty_agensex = df_drugs_sub[(df_drugs_sub["Age"] == " ") & (df_drugs_sub["Sex"] == " ")].shape[0]
print("Jumlah baris data dengan nilai atribut Age dan Sex yang kosong adalah", cnt_empty_agensex)

## 4. Prapemrosesan

...

Tahap pertama pemrosesan adalah mengubah nilai pada kolom `ListActiveIngredient` ke dalam bentuk _one-hot encoding vector_. _One-hot encoding_ mengubah fitur kategorikal/nominal ke format yang lebih sesuai digunakan dalam algoritma klasifikasi dan regresi. Dalam sistem rekomendasi ini, algoritma klasifikasi digunakan sebagai pembentuk model _hybrid_. 

In [ ]:
def active_ingredient_text_cleaning(txt_in):
    txt_out = txt_in.replace("[[", "")
    txt_out = txt_out.replace("]]", "")
    list_out = txt_out.split("], [")
    list_out = [[j.strip("\'") for j in i.split(",")][0].lower() for i in list_out]
    return list_out

list_active_ingredients = []
for i in df_drugs_sub['ListActiveIngredient']:
    list_active_ingredients.append(active_ingredient_text_cleaning(i))

# hapus kolom list active ingredient
df_drugs_sub = df_drugs_sub.drop(columns = ["ListActiveIngredient"])
# buat kolom active ingredients berdasarkan list active ingredients
df_drugs_sub["ActiveIngredients"] = list_active_ingredients

# one-hot encoding
mlb = MultiLabelBinarizer()
df_drugs_sub = df_drugs_sub.join(pd.DataFrame(mlb.fit_transform(df_drugs_sub.pop("ActiveIngredients")),
                                              columns=mlb.classes_,
                                              index=df_drugs_sub.index))

Tahap kedua pemrosesan adalah mengubah nilai pada kolom `ListInactiveIngredient` ke dalam bentuk _one-hot encoding vector_, sama seperti pada `ListActiveIngredient`. 

In [ ]:
def inactive_ingredient_text_cleaning(txt_in):
    txt_out = txt_in.replace("[[", "")
    txt_out = txt_out.replace("]]", "")
    list_out = txt_out.split("], [")
    list_out = [[j.strip("\'") for j in i.split(",")][0].lower() for i in list_out]
    return list_out

list_inactive_ingredients = []
for i in df_drugs_sub['ListInactiveIngredient']:
    list_inactive_ingredients.append(inactive_ingredient_text_cleaning(i))

# hapus kolom list inactive ingredient
df_drugs_sub = df_drugs_sub.drop(columns = ["ListInactiveIngredient"])
# buat kolom inactive ingredients berdasarkan list inactive ingredients
df_drugs_sub["InactiveIngredients"] = list_inactive_ingredients

# one-hot encoding
mlb = MultiLabelBinarizer()
df_drugs_sub = df_drugs_sub.join(pd.DataFrame(mlb.fit_transform(df_drugs_sub.pop("InactiveIngredients")),
                                              columns=mlb.classes_,
                                              index=df_drugs_sub.index))

Tahap ketiga pemrosesan adalah mengubah nilai pada kolom `sides` ke dalam bentuk _one-hot encoding vector_, sama seperti pada `ListActiveIngredient` dan `ListInactiveIngredient`. 

In [ ]:
def sides_text_cleaning(txt_in):
    # ganti whitespace lebih dari 1 menjadi 1 
    txt_out = re.sub('\s+', ' ', txt_in)
    # hapus kalimat keterangan
    txt_out = txt_out.replace("If any of these effects persist or worsen, tell your doctor or pharmacist promptly.", "")
    txt_out = txt_out.replace("If any of these effects last or get worse, tell your doctor or pharmacist promptly.", "")
    txt_out = txt_out.replace("may occur, especially during the first 2 hours after you take the medication .", "")
    txt_out = txt_out.replace("may occur as your body adjusts to this medication .", "")
    txt_out = txt_out.replace("may occur as your body adjusts to the medication .", "")
    txt_out = txt_out.replace("may also occur.", "")
    txt_out = txt_out.replace("may occur.", "")
    txt_out = txt_out.replace("and ", "")
    txt_out = txt_out.replace("your ", "")
    txt_out = txt_out.replace("or ", "")
    return txt_out

list_sides = []
for id, val in df_drugs_sub["Sides"].iteritems():
    # pembersihan text sides dan tokenisasi text sides
    list_sides.append([i.strip().lower() for i in sides_text_cleaning(val).split(",")])

# hapus kolom sides yang lama
df_drugs_sub = df_drugs_sub.drop(columns = ["Sides"])
# buat kolom sides yang baru berdasarkan list sides
df_drugs_sub["Sides"] = list_sides

# one-hot encoding
mlb = MultiLabelBinarizer()
df_drugs_sub = df_drugs_sub.join(pd.DataFrame(mlb.fit_transform(df_drugs_sub.pop("Sides")),
                                              columns=mlb.classes_,
                                              index=df_drugs_sub.index))

Pada `df_drugs` terdapat 1.051 baris data dengan nilai atribut `Age` dan `Sex` yang kosong. Oleh karena itu, dilakukan imputasi dengan menggunakan k-Nearest Neigbors.

In [ ]:
# label encoding map untuk nilai pada sex
map_sex = {"Male":0, "Female":1}
# label encoding map untuk nilai pada age
map_age = {'0-2':0,
           '13-18':1,
           '19-24':2,
           '25-34':3,
           '3-6':4,
           '35-44':5,
           '45-54':6,
           '55-64':7,
           '65-74':8,
           '7-12':9,
           '75 or over':10}

# label encoding pada sex
df_user["Sexmap"] = df_user.Sex.map(map_sex)
# label encoding pada age
df_user["Agemap"] = df_user.Age.map(map_age)

# inverted label encoding map untuk nilai pada sex
inv_map_sex = {v: k for k, v in map_sex.items()}
# inverted label encoding map untuk nilai pada age
inv_map_age = {v: k for k, v in map_age.items()}

# penanganan missing values pada sex dan age
imp = KNNImputer(n_neighbors=1)
mat = imp.fit_transform(df_user[["Agemap", "Sexmap"]])
df_user = pd.concat([df_user, pd.DataFrame(mat, columns=["NewAge", "NewSex"])], axis=1)
df_user["Sex"] = df_user.NewSex.map(inv_map_sex)
df_user["Age"] = df_user.NewAge.map(inv_map_age)
df_user = df_user.drop(columns = ["Agemap", "Sexmap", "NewAge", "NewSex"])

In [ ]:
# df_drugs_sub = df_drugs_sub.drop(df_drugs_sub[(df_drugs_sub["Age"] == " ") & (df_drugs_sub["Sex"] == " ")].index)

### Pembagian set data

Untuk menerapkan _item-based collaborative filtering_ pada sistem rekomendasi obat, dibutuhkan matriks pengguna/obat. Pada matriks tersebut terdapat _"user id"_, _"drug id"_, dan _"drug rating"_. Ilustrasi matriks tersebut adalah sebagai berikut.

|  | drug<sub>1</sub> | drug<sub>2</sub> | drug<sub>3</sub> |
| :---: | :---: | :---: | :---: |
| user<sub>1</sub> | rating<sub>user<sub>1</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>1</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>1</sub>,drug<sub>3</sub></sub> |
| user<sub>2</sub> | rating<sub>user<sub>2</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>2</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>2</sub>,drug<sub>3</sub></sub> |
| user<sub>3</sub> | rating<sub>user<sub>3</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>3</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>3</sub>,drug<sub>3</sub></sub> |
| user<sub>4</sub> | rating<sub>user<sub>4</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>4</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>4</sub>,drug<sub>3</sub></sub> |
| user<sub>5</sub> | rating<sub>user<sub>5</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>5</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>5</sub>,drug<sub>3</sub></sub> |


_"user id"_ tidak tersedia dalam `df_drugs`. Oleh karena itu, ....

_"drug id"_ tersedia dalam `df_drugs`. Oleh karena itu, ....

_"drug rating"_ tidak tersedia dalam `df_drugs`. Oleh karena itu, ....

In [ ]:
df_drugs.columns

In [ ]:
# Data Drug
df_item = pd.DataFrame({'Drug':df_drugs['Drug'],
                        'ListActiveIngredient':df_drugs['ListActiveIngredient'],
                        'ListInactiveIngredient':df_drugs['ListInactiveIngredient'],
                        'Sides':df_drugs['Sides']})

df_item = df_item.drop_duplicates().reset_index(drop=True)
df_item["ItemId"] = df_item.index + 1
df_item = df_item[['ItemId','Drug','ListActiveIngredient','ListInactiveIngredient','Sides']]
df_item.head()

In [ ]:
# Data User
df_user = pd.DataFrame({'Age':df_drugs['Age'], 'Sex':df_drugs['Sex'], 'Condition':df_drugs['Condition']})
df_user = df_user.drop_duplicates().reset_index(drop=True)
df_user["UserId"] = df_user.index + 1
df_user = df_user[['UserId','Age','Sex','Condition']]
df_user.head()

In [ ]:
# Data Rating
df_tmp = df_drugs.merge(df_item[['ItemId','Drug','ListActiveIngredient','ListInactiveIngredient','Sides']], 'left')
df_rating = df_tmp.merge(df_user[['UserId','Age','Sex','Condition']], 'left')
df_rating = df_rating[['UserId','ItemId','EaseofUse','Effectiveness','Satisfaction']]
df_rating.head()

Prapemrosesan pada atribut `sex` dan `age` dalam `df_user`. 

---

## 5. Pembangunan Model Rekomendasi

...


In [ ]:
# 70% baris data dipilih secara random 
# sebagai data train dan sisanya sebagai data test
df_rating_train = df_rating.sample(frac = 0.7)
df_rating_test = df_rating.drop(df_rating_train.index)

In [ ]:
x_train = df_rating_train[['UserId', 'ItemId']]
y_train = df_rating_train[['Effectiveness']]
x_test = df_rating_test[['UserId', 'ItemId']]
y_test = df_rating_test['Effectiveness']

In [ ]:
sides = df_item['Sides'].to_list()
condition_col_names = list(set(df_user['Condition'].to_list()))

# get one hot encoding of column condition
one_hot = pd.get_dummies(df_user['Condition'])
# drop column Condition as it is now encoded
df_user = df_user.drop('Condition', axis = 1)
# join the encoded df_user
df_user = df_user.join(one_hot)

# get one hot encoding of column sex
one_hot = pd.get_dummies(df_user['Sex'])
# drop column Sex as it is now encoded
df_user = df_user.drop('Sex', axis = 1)
# join the encoded df_user
df_user = df_user.join(one_hot)

df_item = df_item[['ItemId','Sides']]

# Next we merge the movie and user data with our train and test dataset
x_train = x_train.join(df_user.set_index('UserId'),
                       on = 'UserId').join(df_item.set_index('ItemId'),
                                           on = 'ItemId')
x_test = x_test.join(df_user.set_index('UserId'),
                     on = 'UserId').join(df_item.set_index('ItemId'),
                                         on = 'ItemId')

In [ ]:
df_user[["Age"]]

In [ ]:
# define ordinal encoding
encoder = preprocessing.OrdinalEncoder()
# transform data
result = encoder.fit_transform(df_user[["Age"]])

In [ ]:
result

In [ ]:
x_train.head()

### Recommendation Model Setting

In [ ]:
model1 = xgb.XGBRegressor(objective='reg:squarederror')
model1.fit(x_train, y_train)

pred1 = model1.predict(x_test)
rmse = np.sqrt(np.mean((pred1 - y_test.to_numpy())**2))
print(f'content-based rmse = {rmse}')

## 6. Pengujian Model Rekomendasi

...

---